In [8]:
pip install selenium


                                              0.0/10.2 MB ? eta -:--:--
                                              0.2/10.2 MB 6.3 MB/s eta 0:00:02
     --                                       0.6/10.2 MB 6.7 MB/s eta 0:00:02
     ----                                     1.0/10.2 MB 7.3 MB/s eta 0:00:02
     --------                                 2.2/10.2 MB 11.6 MB/s eta 0:00:01
     ------------                             3.2/10.2 MB 13.6 MB/s eta 0:00:01
     -----------------                        4.5/10.2 MB 16.8 MB/s eta 0:00:01
     --------------------                     5.3/10.2 MB 16.9 MB/s eta 0:00:01
     ------------------------                 6.3/10.2 MB 18.5 MB/s eta 0:00:01
     ----------------------------             7.3/10.2 MB 18.7 MB/s eta 0:00:01
     -------------------------------          7.9/10.2 MB 18.2 MB/s eta 0:00:01
     ----------------------------------       8.7/10.2 MB 17.9 MB/s eta 0:00:01
     -------------------------------------    9.5/


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/10.6 MB ? eta -:--:--
                                              0.1/10.6 MB 2.6 MB/s eta 0:00:05
     -                                        0.4/10.6 MB 5.1 MB/s eta 0:00:02
     ---                                      1.0/10.6 MB 7.0 MB/s eta 0:00:02
     ------                                   1.6/10.6 MB 8.7 MB/s eta 0:00:02
     ---------                                2.5/10.6 MB 10.5 MB/s eta 0:00:01
     -----------                              3.1/10.6 MB 11.7 MB/s eta 0:00:01
     --------------                           3.9/10.6 MB 12.6 MB/s eta 0:00:01
     ------------------                       4.9/10.6 MB 14.1 MB/s eta 0:00:01
     --------------------                     5.6/10.6 MB 14.2 MB/s eta 0:00:01
     ------------------------                 6.5/10.6 MB 15.3 MB/s eta 0:00:01
     ----------------------------             7.5/10.6 MB 16.0 MB/s eta 0:00:01
     --------------------------------         8.7/1

In [11]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
print ("Selenium webdriver Version: %s" % (webdriver.__version__))

Selenium webdriver Version: 4.15.1


In [12]:
def init_scrapper(driverpath,keyword,headless=False):
    #""""initialize the glassdor scrapper given a job keyword"""
    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # if you'd like to scrape without a new Chrome window every time.
    if headless: options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    #driver = webdriver.Chrome(executable_path=driverpath, options=options)
    cService = webdriver.ChromeService(driverpath)
    driver=webdriver.Chrome(service=cService, options=options)
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)
    #while(True):
    #   pass
    return driver

In [13]:
def get_companyname_rating(company_name):
    #"""gets the company name and (if exist) gets the company rating too"""
    splited = company_name.split("\n")
    # company name comes with rating
    if len(splited) > 1:
        return splited[0],  splited[1]
    # just company name
    else:
        return company_name, -1

In [14]:
def get_company_overview_data(Company_Overview):
    #""""gets the company overview data (dont the name of the func speaks for itself?)"""
    Company_Overview_dict = {"size":"",
                            "founded":"",
                            "type":"",
                            "industry":"",
                            "sector":"",
                            "revenue":"",}

    for i, key in enumerate(Company_Overview_dict):
        try:
            Company_Overview_dict[key] = Company_Overview[i].text
        except:
            # print(f"{key} not found")
            Company_Overview_dict[key] = -1
    return Company_Overview_dict

In [15]:
path="C:\\Users\Emmanuel\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe" #route to the chromedriver
keyword="blockchain" #what do you want to find on the webpage
driver = init_scrapper(path,keyword)

In [ ]:
jobs=[]
jobs = driver.find_element(By.CLASS_NAME,'JobDetails_jobTitle__Rw_gn').text
jobs

In [16]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time, failsafe = False):

    #"""Gathers jobs as a dataframe, scraped from Glassdoor"""
    driver = init_scrapper(path,keyword)
    jobs = []
    #Let the page load. Change this number based on your internet speed.
    time.sleep(2*slp_time)
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, 'JobsList_jobListItem__JBBUV')  #react-job-listing for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:

            # print(f"Progress: {len(jobs)}/{num_jobs}")
            if len(jobs) >= num_jobs:
                break
            job_button.click()  #You might
            time.sleep(slp_time)
            # Test for the "Sign Up" prompt and get rid of it.
            # close modal (just appear on the first pass)
            if len(jobs) == 1:
                try:
                    driver.find_element(By.CLASS_NAME, 'CloseButton').click()
                    print("modal cerrado con exito")
                except NoSuchElementException:
                    print("no hay modal xd")

            collected_successfully = False
            while not collected_successfully:
                try:
                    # En los siguientes textos vacíos vamos a rellenarlos con las clases de los
                    # elementos HTML que nos interese para recuperar la información
                    company_name_ = driver.find_element(By.CLASS_NAME, 'EmployerProfile_profileContainer__d5rMb').text
                    #sometimes company name comes with the rating, if its the case get_companyname_rating handles it
                    company_name, rating = get_companyname_rating(company_name_)
                    location = driver.find_element(By.CLASS_NAME,'JobDetails_location__MbnUM').text
                    job_title = driver.find_element(By.CLASS_NAME,'JobDetails_jobTitle__Rw_gn').text
                    job_description = driver.find_element(By.CLASS_NAME,'JobDetails_jobDescription__6VeBn.JobDetails_blurDescription__fRQYh').text
                    #salary_estimate = driver.find_element(By.CLASS_NAME,'SalaryEstimate_averageEstimate__xF_7h').text
                    collected_successfully = True
                    print("job info collected successfully :)")
                except:
                    time.sleep(slp_time)
                    print("job info error, trying again...")
                    break

            job_data_dict = {"Job Title" : job_title,
                            "Company Name" : company_name,
                            "Location" : location,
                            #"Salary Estimate" : salary_estimate,
                            "Rating" : rating,
                            "Job Description" : job_description,
                        }

            #Printing for debugging
            if verbose:
                print(job_data_dict)

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                Company_Overview = driver.find_elements(By.CLASS_NAME,'JobDetails_overviewItemValue__5TqNi')
                company_overview_dict = get_company_overview_data(Company_Overview)

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                company_overview_dict = {"size":-1,
                                        "founded":-1,
                                        "type":-1,
                                        "industry":-1,
                                        "sector":-1,
                                        "revenue":-1}

            if verbose:
                print(company_overview_dict)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            job_data_dict.update(company_overview_dict)

            jobs.append(job_data_dict)

            #add job to jobs


        #Clicking on the "next page" button
        try:
            driver.find_element(By.CLASS_NAME,'nextButton').click()
            time.sleep(slp_time)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        jobs
    driver.close()
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [17]:
path = "C:\\Users\Emmanuel\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
keyword = "Data science"
num_jobs = 5
slp_time = 6
verbose = True
failsafe = True

df = get_jobs(keyword, num_jobs, verbose, path, slp_time,failsafe)
df

job info collected successfully :)
{'Job Title': 'Compliance & METRC Administrator', 'Company Name': 'Lake Co Development', 'Location': 'Lower Lake, CA', 'Rating': -1, 'Job Description': 'Alchemy 29 is a vertically integrated company made up of leading industry experts with solid track records of success within the Cannabis industry. Our core values lie in creating a culture of empowerment by giving team members the tools, training, and autonomy to excel in their chosen path. We work hard to support our staff by creating good jobs with competitive wages and continually look for ways to affect positive change within our operational family.\nAlchemy 29 is comprised of four multi-acre farms, our teams employ sustainable cultivation practices including, good agricultural practices (GAP) in all of our farming operations. We offer a full suite of cost-conscious manufacturing and processing options, white-labeling services, and our full line of branded products from our pharmaceutical-grade l

,Job Title,Company Name,Location,Rating,Job Description,size,founded,type,industry,sector,revenue
0,Compliance & METRC Administrator,Lake Co Development,"Lower Lake, CA",-1,Alchemy 29 is a vertically integrated company ...,-1,-1,-1,-1,-1,-1
1,Master Electrician - Data Center/Bitcoin Minin...,Lake Co Development,"Bruceton Mills, WV",-1,Alchemy 29 is a vertically integrated company ...,-1,-1,-1,-1,-1,-1
2,Pre- Litigation Attorney,Lake Co Development,"Phoenix, AZ",-1,Pre-Litigation Attorney – Esquire Law\nAbout E...,51 to 200 Employees,--,Company - Public,--,--,Unknown / Non-Applicable
3,Data Architect IV,Lake Co Development,"Belcamp, MD",-1,DCI Job Requirement for:\nData Architect/Scien...,1 to 50 Employees,--,Contract,--,--,Unknown / Non-Applicable
4,Senior Agriculture/Forestry (AFOLU) GHG Accoun...,Lake Co Development,"Washington, DC",-1,Position Description:\n\nMathematica applies e...,1001 to 5000 Employees,1986,Company - Private,Research & Development,Management & Consulting,$100 to $500 million (USD)


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
